# Finetuning de Llama-3.1 8B

Aqui tratamos de afinar el modelo Llama-3.1 8B utilizando el conjunto de datos Alpaca de Yahma, disponible en Hugging Face. Este conjunto de datos es una versión filtrada y limpia del conjunto original de Alpaca, creado por investigadores de la Universidad de Stanford.

**Tools:**
* El conjunto de datos Alpaca es de propósito general y contiene aproximadamente 52,000 pares de instrucciones y respuestas, abarcando una amplia gama de temas y tareas como preguntas generales, escritura, resolución de problemas y explicaciones. No está enfocado en un área específica, sino que tiene como objetivo mejorar la capacidad del modelo para seguir instrucciones y generar respuestas coherentes en diversos contextos

* La librería **Unsloth**, lo que permite entrenar el modelo de manera más rápida y eficiente en un entorno como Google Colab.

**Objetivos:**

1. **Instalación de Unsloth:**
   - Se instala y actualiza la librería Unsloth, esencial para acelerar el entrenamiento de modelos grandes.
   - Unsloth proporciona optimizaciones que permiten entrenar modelos con menos uso de memoria y mayor velocidad.

2. **Carga del modelo y tokenizador:**
   - Se definen parámetros clave como la longitud máxima de secuencia y el tipo de datos (por ejemplo, Float16 o BFloat16).
   - Se carga el modelo pre-entrenado Llama-3.1 8B y su tokenizador asociado utilizando la función proporcionada por Unsloth.
   - Se habilita la opción de cargar el modelo en 4 bits para reducir el uso de memoria (cuantización en 4 bits).

3. **Configuración de adaptadores LoRA:**
   - Se aplican adaptadores **LoRA** (Low-Rank Adaptation) al modelo, lo que permite entrenar solo una pequeña parte de los parámetros (entre 1% y 10%), reduciendo significativamente el tiempo y recursos necesarios.
   - Se configuran los hiperparámetros de LoRA, como el rango `r`, `lora_alpha` y `lora_dropout`, y se especifican los módulos del modelo que serán ajustados.

4. **Preparación del conjunto de Datos:**
   - Se utiliza el conjunto de datos **Alpaca** de Yahma, una versión filtrada y limpia del original, ideal para tareas de ajuste fino.
   - Se define un formato de prompt personalizado que incluye una instrucción, una entrada opcional y un espacio para la respuesta.
   - Se procesa el conjunto de datos para adaptarlo al formato requerido, asegurándose de añadir el token de fin de secuencia (**EOS_TOKEN**) para evitar generaciones infinitas durante la inferencia.

5. **Entrenamiento del modelo:**
   - Se configura el entrenador utilizando el **SFTTrainer** de la librería **TRL** (Training Reward Learning).
   - Se establecen argumentos de entrenamiento como el tamaño de lote, pasos de calentamiento, tasa de aprendizaje, optimizador y otros hiperparámetros relevantes.
   - Se inicia el proceso de entrenamiento, durante el cual se monitorean y registran estadísticas de memoria y tiempo para evaluar el rendimiento y eficiencia.

6. **Generación de respuestas (Inferencia):**
   - Se habilita el modo de inferencia optimizada proporcionado por Unsloth para mejorar la velocidad de generación.
   - Se proporciona un ejemplo de instrucción y entrada, y se genera una respuesta utilizando el modelo ajustado.
   - Se demuestra cómo el modelo puede ser utilizado para generar respuestas coherentes y relevantes basadas en las instrucciones dadas.

7. **Guardado del modelo ajustado:**
   - Se guardan los adaptadores LoRA resultantes del entrenamiento, lo que permite cargar y utilizar el modelo ajustado en futuras sesiones sin necesidad de reentrenamiento.
   - Se ofrece la opción de cargar estos adaptadores para realizar inferencia en otros entornos o compartirlos con la comunidad.



- **Optimizados:**
  - Se aprovechan las optimizaciones de Unsloth, como la cuantización en 4 bits y el uso de gradient checkpointing, para reducir el uso de memoria y acelerar el entrenamiento.
  - Se ajustan dinámicamente parámetros como el tipo de datos (`fp16` o `bf16`) en función del hardware disponible, garantizando compatibilidad y rendimiento óptimo.


In [ ]:
# Instalación de Unsloth
!pip install unsloth
# Instalamos la última versión de Unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


Found existing installation: unsloth 2024.11.7
Uninstalling unsloth-2024.11.7:
  Successfully uninstalled unsloth-2024.11.7
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-h8vzzajg/unsloth_49a8c48aa12841629d81fd56a4412c1c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-h8vzzajg/unsloth_49a8c48aa12841629d81fd56a4412c1c
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.7-py3-none-any.whl size=163138 sha256=8f49fa0959e114834ad2415d799bd7d1811a5d814221ba83e5b2f4d7a07b9ee2
  Stored in directory: /tmp/pip-ephem-wheel-cache-6jzzz37x/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [ ]:
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
import os
os.environ["WANDB_DISABLED"] = "true"

# Definición de variables globales
MAX_SEQ_LENGTH = 2048  # Longitud máxima de secuencia. Se puede ajustar según necesidad.
DTYPE = None  # Tipo de datos. None para detección automática, Float16 para Tesla T4/V100, BFloat16 para Ampere+.
LOAD_IN_4BIT = True  # Cargar el modelo en 4 bits para reducir uso de memoria.

# Lista de modelos pre-cuantizados en 4 bits soportados
FOURBIT_MODELS = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
    # Más modelos disponibles en https://huggingface.co/unsloth
]



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
def load_model(model_name, max_seq_length, dtype, load_in_4bit):
    #Carga el modelo y el tokenizador usando FastLanguageModel de Unsloth.
    # model_name: Nombre del modelo a cargar.
    # max_seq_length: Longitud máxima de secuencia.
    # dtype: Tipo de dato (None para detección automática).
    # load_in_4bit: Booleano para cargar en 4 bits.

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
        # token="hf_...",  # Usar si se requieren tokens de autenticación
    )
    return model, tokenizer


In [ ]:
def configure_lora(model, r=16, lora_alpha=16, lora_dropout=0, use_gc="unsloth", random_state=3407):
    """
    Configura los adaptadores LoRA para el modelo.

    Parámetros:
    - model: El modelo al que se aplicarán los adaptadores.
    - r: Rango de LoRA (sugerido 8, 16, 32, 64, 128).
    - lora_alpha: Hiperparámetro de LoRA.
    - lora_dropout: Dropout para LoRA (0 es óptimo para esta configuración).
    - use_gc: Uso de gradient checkpointing ("unsloth" para optimización).
    - random_state: Semilla para reproducibilidad.

    Retorna:
    - model: Modelo con los adaptadores LoRA configurados.
    """
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"]
    model = FastLanguageModel.get_peft_model(
        model,
        r=r,
        target_modules=target_modules,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias="none",  # "none" es óptimo para esta configuración
        use_gradient_checkpointing=use_gc,
        random_state=random_state,
        use_rslora=False,  # No usamos Rank Stabilized LoRA en este caso
        loftq_config=None,  # No usamos LoftQ
    )
    return model


In [ ]:
def prepare_data(tokenizer, dataset_name="yahma/alpaca-cleaned", split="train"):
    """
    Prepara el conjunto de datos para el entrenamiento con flexibilidad en las claves de entrada.

    Parámetros:
    - tokenizer: El tokenizador del modelo.
    - dataset_name: Nombre del conjunto de datos a utilizar.
    - split: División del conjunto de datos a cargar.

    Retorna:
    - dataset: Conjunto de datos procesado y tokenizado.
    """
    # Definimos el formato del prompt
    default_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    poem_prompt = """Below is a poem. Write a response that analyzes or interprets its meaning.

### Poem:
{}

### Response:
{}"""

    EOS_TOKEN = tokenizer.eos_token  # Añadimos el token EOS

    def formatting_prompts_func(examples):
        # Verificar si las claves estándar existen
        if "instruction" in examples and "input" in examples and "output" in examples:
            instructions = examples["instruction"]
            inputs = examples["input"]
            outputs = examples["output"]
            texts = [
                default_prompt.format(instruction, input_text, output) + EOS_TOKEN
                for instruction, input_text, output in zip(instructions, inputs, outputs)
            ]
        else:
            # Para el dataset de poemas con solo 'poem'
            poems = examples["poem"]
            texts = [
                poem_prompt.format(poem, "") + EOS_TOKEN for poem in poems
            ]
        return {"text": texts}

    # Cargamos y procesamos el conjunto de datos
    dataset = load_dataset(dataset_name, split=split)
    dataset = dataset.map(formatting_prompts_func, batched=True)
    return dataset


In [ ]:
def print_dataset_examples(dataset_name="xaviviro/FEDERICO-GARCIA-LORCA-canciones-poemas-romances", split="train", num_examples=5):
    """
    Imprime ejemplos de un dataset, adaptándose automáticamente a las claves disponibles.

    Parámetros:
    - dataset_name: Nombre del dataset en Hugging Face.
    - split: División del dataset a cargar (e.g., 'train').
    - num_examples: Número de ejemplos a mostrar.
    """
    from datasets import load_dataset

    # Cargar el dataset
    dataset = load_dataset(dataset_name, split=split)

    # Mostrar ejemplos
    print(f"Mostrando {num_examples} ejemplos del dataset '{dataset_name}' ({split} split):\n")
    for i in range(num_examples):
        example = dataset[i]
        print(f"Ejemplo {i+1}:")

        # Adaptar según las claves disponibles
        if "instruction" in example and "input" in example and "output" in example:
            print("Instruction:", example["instruction"])
            print("Input:", example["input"])
            print("Output:", example["output"])
        elif "poem" in example:
            print("Poem:", example["poem"])
        else:
            print("Formato desconocido. Claves disponibles:", example.keys())

        print("-" * 50)

# Llamada de ejemplo con el dataset de Alpaca
#print_dataset_examples(dataset_name="bertin-project/alpaca-spanish")

# Llamada de ejemplo con el dataset de poemas
#print_dataset_examples(dataset_name="xaviviro/FEDERICO-GARCIA-LORCA-canciones-poemas-romances")


In [ ]:
def train_model(model, tokenizer, dataset, max_seq_length):
    """
    Entrena el modelo utilizando SFTTrainer.

    Parámetros:
    - model: El modelo a entrenar.
    - tokenizer: El tokenizador asociado.
    - dataset: Conjunto de datos para entrenamiento.
    - max_seq_length: Longitud máxima de secuencia.

    Retorna:
    - trainer_stats: Estadísticas del entrenamiento.
    """
    # Determinamos si usar fp16 o bf16
    fp16 = not is_bfloat16_supported()
    bf16 = is_bfloat16_supported()

    training_args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=1,  # Descomentar para un entrenamiento completo
        max_steps=60,  # Limitamos a 60 pasos para acelerar pruebas
        learning_rate=2e-4,
        fp16=fp16,
        bf16=bf16,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,  # Puede acelerar el entrenamiento para secuencias cortas
        args=training_args,
    )

    # Obtenemos estadísticas iniciales de la GPU
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
    max_memory = round(gpu_stats.total_memory / 1024**3, 3)
    print(f"GPU = {gpu_stats.name}. Memoria máxima = {max_memory} GB.")
    print(f"{start_gpu_memory} GB de memoria reservada al inicio.")

    # Entrenamos el modelo
    trainer_stats = trainer.train()

    # Estadísticas de memoria y tiempo tras el entrenamiento
    used_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
    print(f"{trainer_stats.metrics['train_runtime']} segundos utilizados para el entrenamiento.")
    print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutos en total.")
    print(f"Memoria máxima reservada = {used_memory} GB.")
    print(f"Memoria usada para entrenamiento = {used_memory_for_lora} GB.")
    print(f"Porcentaje de memoria usada = {used_percentage} %.")
    print(f"Porcentaje de memoria usada para entrenamiento = {lora_percentage} %.")

    return trainer_stats


In [ ]:
def generate_response(model, tokenizer, instruction, input_text="", max_new_tokens=64):
    """
    Genera una respuesta dada una instrucción y un input opcional.

    Parámetros:
    - model: El modelo para generación.
    - tokenizer: El tokenizador asociado.
    - instruction: Instrucción para el modelo.
    - input_text: Texto de entrada adicional.
    - max_new_tokens: Número máximo de tokens a generar.

    Retorna:
    - response: Texto generado por el modelo.
    """
    # Formato del prompt
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
    # Preparamos la entrada
    inputs = tokenizer(
        [alpaca_prompt.format(instruction, input_text, "")],
        return_tensors="pt"
    ).to("cuda")

    # Habilitamos la inferencia optimizada
    FastLanguageModel.for_inference(model)

    # Generamos la respuesta
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response


In [ ]:
def save_model(model, tokenizer, save_directory="lora_model"):
    #Guarda los adaptadores LoRA del modelo.
    # model: El modelo entrenado con adaptadores LoRA.
    # tokenizer: El tokenizador asociado al modelo.
    # save_directory: Directorio donde se guardará el modelo.

    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)
    print(f"Modelo y tokenizador guardados en {save_directory}")


In [ ]:
# Carga del modelo y tokenizador
model_name = "unsloth/Meta-Llama-3.1-8B"  # Puede cambiarse por otro modelo de FOURBIT_MODELS
model, tokenizer = load_model(
    model_name=model_name,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT
)

# Configuración de adaptadores LoRA
model = configure_lora(model)

# Preparación de los datos
dataset = prepare_data(tokenizer)

# Entrenamiento del modelo
trainer_stats = train_model(model, tokenizer, dataset, MAX_SEQ_LENGTH)

# Ejemplo de inferencia
instruction = "Continúa la secuencia de Fibonacci."
input_text = "1, 1, 2, 3, 5, 8"
response = generate_response(model, tokenizer, instruction, input_text)
print("Respuesta generada:")
print(response)

# Guardamos el modelo entrenado
save_model(model, tokenizer)


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


GPU = Tesla T4. Memoria máxima = 14.748 GB.
5.984 GB de memoria reservada al inicio.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.586800
2,2.115200
3,1.672900
4,1.863000
5,1.678900
6,1.490100
7,1.080900
8,1.270000
9,1.144400
10,1.122400


493.4542 segundos utilizados para el entrenamiento.
8.22 minutos en total.
Memoria máxima reservada = 7.924 GB.
Memoria usada para entrenamiento = 1.94 GB.
Porcentaje de memoria usada = 53.729 %.
Porcentaje de memoria usada para entrenamiento = 13.154 %.
Respuesta generada:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continúa la secuencia de Fibonacci.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765
Modelo y tokenizador guardados en lora_model


In [ ]:
# Esto deberia vaciar la memmoria de la GPU, pero google colab mantiene algunas caches ocultas
import torch

# Elimina todos los tensores de la GPU
torch.cuda.empty_cache()
# Elimina todos los tensores de la GPU
torch.cuda.empty_cache()

In [ ]:
# Ejemplo de inferencia
instruction = "Continúa la secuencia de Fibonacci."
input_text = "1, 1, 2, 3, 5, 8, 13"
response = generate_response(model, tokenizer, instruction, input_text)
print("Respuesta generada:")
print(response)

Respuesta generada:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continúa la secuencia de Fibonacci.

### Input:
1, 1, 2, 3, 5, 8, 13

### Response:
Continuando la secuencia de Fibonacci, los siguientes números serían: 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 177


In [ ]:
### TODO 1: Utiliza diferentes dataset en español:
# bertin-project/alpaca-spanish
# xaviviro/FEDERICO-GARCIA-LORCA-canciones-poemas-romances

### TODO 2: Cambiar el tamaño del lote si la memoria de la GPU lo permite.

### TODO 3: Ajustar los pasos de acumulación de gradientes según la capacidad de hardware.

### TODO 4: Modificar warmup_steps para experimentos con diferentes curvas de aprendizaje.

### TODO 5: Cambiar num_train_epochs para entrenamientos completos.

### TODO 6: Ajustar max_steps para pruebas rápidas o entrenamientos largos.

### TODO 7: Modificar learning_rate según el optimizador o tamaño del modelo.

### TODO 8: Ajustar logging_steps para monitorear el entrenamiento con mayor o menor frecuencia.

### TODO 9: Cambiar optim a un optimizador diferente si es necesario.

### TODO 10: Ajustar weight_decay para diferentes niveles de regularización.

### TODO 11: Cambiar lr_scheduler_type para probar diferentes estrategias de decaimiento.

### TODO 12: Establecer seed para reproducibilidad si se requiere.

### TODO 13: Cambiar output_dir para guardar los resultados en otro directorio.

### TODO 14: Ajustar max_seq_length según la longitud esperada de las secuencias en el dataset.

### TODO 15: Revisar dataset_num_proc para paralelismo óptimo en la tokenización.
